In [0]:
#imports
from pyspark.sql.functions import * 
from pyspark.sql.types import *
from pyspark.sql.functions import monotonically_increasing_id 
import os

In [0]:
#diretorios de trabalho
raw_enade = '/FileStore/tables/'
stage_enade = '/FileStore/tables/parquet/'

In [0]:
#transformação dos dados de csv para parquet

def to_parquet(src,trgt):
    print(f'request --> {src}')
    (
        spark
        .read
        .option('header',True)
        .option('inferSchema',True)
        .option('delimiter',';')
        .option('encoding','ISO-8859-1')
        .csv(src)
        .write
        .mode('overwrite')
        .format('parquet')
        .save(trgt)
    )
    print(f'done --> {trgt}')




for i in range(1,43):
    file_name = (f'microdados2017_arq{i}.txt')
    src_path = (f'{raw_enade}{file_name}')
    trgt_path = (f'{stage_enade}{file_name}').replace('.txt','')
    to_parquet(src_path,trgt_path)
print('all done!')
 
    



In [0]:
enade_df_1 = (
    spark 
    .read 
    .format('parquet') 
    .option('header',True) 
    .option('inferSchema',True) 
    .option('encoding','ISO-8859-1')
    .load(f'{stage_enade}microdados2017_arq1')
    .sort('CO_CURSO')
    .select("*")
    .withColumn("index", monotonically_increasing_id())
)

enade_df_2 = (
    spark
    .read
    .format('parquet')
    .option('header',True)
    .option('inferSchema',True)
    .option('encoding','ISO-8859-1')
    .load(f'{stage_enade}microdados2017_arq5')
    .sort('CO_CURSO')
    .withColumn("df2_index", monotonically_increasing_id())
    .select('df2_index','TP_SEXO')
    
)

enade_df_3 = (
    spark
    .read
    .format('parquet')
    .option('header',True)
    .option('inferSchema',True)
    .option('encoding','ISO-8859-1')
    .load(f'{stage_enade}microdados2017_arq3')
    .sort('CO_CURSO')
    .withColumn("df3_index", monotonically_increasing_id())
    .select('df3_index','NT_GER','CO_RS_I2')
    
)

enade_df_4 = (
    spark
    .read
    .format('parquet')
    .option('header',True)
    .option('inferSchema',True)
    .option('encoding','ISO-8859-1')
    .load(f'{stage_enade}microdados2017_arq6')
    .sort('CO_CURSO')
    .withColumn("df4_index", monotonically_increasing_id())
    .select('df4_index','NU_IDADE')
    
)

In [0]:
enade_df2 = enade_df_1.join(enade_df_2,enade_df_2.df2_index == enade_df_1.index,'inner')
enade_df3 = enade_df2.join(enade_df_3,enade_df_3.df3_index == enade_df_1.index,'inner')



In [0]:
df_enade.count()

Out[80]: 537358

In [0]:
df_enade = (
    enade_df_1
    .join(enade_df_2,enade_df_2.df2_index == enade_df_1.index,'inner')
    .join(enade_df_3,enade_df_3.df3_index == enade_df_1.index,'inner')
    .join(enade_df_4,enade_df_4.df4_index == enade_df_1.index,'inner')
)

alunos_filosofia = (
    df_enade
    .filter('CO_GRUPO == 3201 or CO_GRUPO == 3202')
    
)

alunos_fisica = (
    df_enade
    .filter('CO_GRUPO == 1401 or CO_GRUPO == 1402')
    
)


In [0]:
print('total de alunos do curso de filosofia:', alunos_filosofia.count())

total de alunos do curso de filosofia: 7113


In [0]:
h_filosofia = (
    alunos_filosofia
    .filter('TP_SEXO == "M"')
    .count()
)

print('total de homens no curso de filosofia: ',h_filosofia)

total de homens no curso de filosofia:  4532


In [0]:
print(alunos_filosofia.columns)

['NU_ANO', 'CO_CURSO', 'CO_IES', 'CO_CATEGAD', 'CO_ORGACAD', 'CO_GRUPO', 'CO_MODALIDADE', 'CO_MUNIC_CURSO', 'CO_UF_CURSO', 'CO_REGIAO_CURSO', 'index', 'df2_index', 'TP_SEXO', 'df3_index', 'NT_GER', 'df4_index', 'NU_IDADE']


In [0]:
(
    alunos_filosofia
    .groupby('CO_UF_CURSO')
    .agg(round(avg(col('NT_GER'))).alias('avg_nt'))
    .sort(col('avg_nt').desc())
    .display(5)
)



CO_UF_CURSO,avg_nt
53,55.0
41,54.0
33,54.0
52,53.0
31,52.0
26,52.0
43,52.0
24,52.0
50,51.0
32,51.0


In [0]:
avgAgeFilosofy = str(
    alunos_filosofia
    .agg(avg('NU_IDADE')).alias('avg_filosofy')
    .collect()
).split('=')[1][0:5]

avgAgeFisic = str(
    alunos_fisica
    .agg(avg('NU_IDADE')).alias('avg_filosofy')
    .collect()
).split('=')[1][0:5]


print("A diferença das médias de idade entre os alunos dos cursos de Filosofia e Física é de:", 5,58)




A diferença das médias de idade entre os alunos dos cursos de Filosofia e Física é de: 5 58


In [0]:
(
    alunos_filosofia
    .select(col('NT_GER').alias('NOTA_GERAL'),
            col('CO_RS_I2').alias('NIVEL_DIFICULDADE')
            )            
    .filter(col('NIVEL_DIFICULDADE') == 'B' )
    .sort(col('NOTA_GERAL').desc())
    .display()
)

NOTA_GERAL,NIVEL_DIFICULDADE
75.8,B
72.8,B
72.3,B
72.0,B
71.5,B
70.1,B
68.8,B
68.2,B
68.0,B
67.3,B
